# Indicators of Heavy Traffic on I-94

In this project, we're going to analyze a dataset about the westbound traffic on the I-94 Interstate highway.

The goal of our analysis is to determine a few indicators of heavy traffic on I-94. These indicators can be weather type, time of the day, time of the week, etc.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

i_94 = pd.read_csv('Metro_Interstate_Traffic_Volume.csv')
i_94.head().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   holiday              5 non-null      object 
 1   temp                 5 non-null      float64
 2   rain_1h              5 non-null      float64
 3   snow_1h              5 non-null      float64
 4   clouds_all           5 non-null      int64  
 5   weather_main         5 non-null      object 
 6   weather_description  5 non-null      object 
 7   date_time            5 non-null      object 
 8   traffic_volume       5 non-null      int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 488.0+ bytes
